In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
import xlrd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMZN', 'ANET',
    'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'DDOG', 'DOCS', 'DOCU',
    'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED', 'GOOG', 'INTG',
    'ISRG', 'JD', 'LSPD', 'LULU', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH',
    'NVCR', 'NVDA', 'OKTA', 'OPEN', 'PANW', 'PAYC', 'PINS', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP',
    'SNOW', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO',
    'TWTR', 'TYL', 'UPST', 'VEEV', 'VLD', 'XPEV', 'ZEN', 'ZM']

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADBE
ADYEY


### E*Trade

In [16]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [17]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,686.08
ABMD,541.36
ABNB,344.28000000000003
ADYEY,5028.2
AMD,2110.9


In [18]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,686
ABMD,541
ABNB,344
ADYEY,5028
AMD,2111


In [19]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [25]:
et_df.loc[et_df.ET == '--']
#et_df.loc['NVCR', 'ET'] = 79.24

,ET


In [26]:
check_symbols(et_df)

All clear


In [27]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,686
ABMD,541
ABNB,344
ADBE,0
ADYEY,5028


### Fidelity

In [28]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Aug-19-2022.csv'

In [29]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Aug-19-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_08_19.xls
mw.csv


In [30]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("06", "07")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,14875.49,$1.00,$0.00,$14875.49,NaN,NaN,NaN,NaN,38.09%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,1.00,$171.52,-$2.63,$171.52,-$2.63,-1.52%,-$2.21,-1.28%,0.44%,$173.73,$173.73,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$270.68,-$7.50,$1082.72,-$30.00,-2.70%,+$64.03,+6.28%,2.77%,$1018.69,$254.67,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,2.00,$114.76,-$6.51,$229.52,-$13.02,-5.37%,-$55.98,-19.61%,0.59%,$285.50,$142.75,Cash
4,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,11.00,$16.22,-$1.403,$178.42,-$14.24,-7.40%,-$7.43,-4.00%,0.46%,$185.85,$16.90,Cash


In [31]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,172
2,ROLLOVER IRA,ABMD,1083
3,ROLLOVER IRA,ABNB,230
4,ROLLOVER IRA,ADYEY,178
5,ROLLOVER IRA,AMD,1631


In [32]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,172
ABMD,1083
ABNB,230
ADYEY,178
AMD,1631


In [33]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [34]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,686,172,8748,0
ABMD,541,1083,2165,0
ABNB,344,230,344,0
ADBE,0,0,0,0
ADYEY,5028,178,2855,114


### TD Ameritrade

In [35]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [36]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'

In [37]:
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [38]:
td = parse_file(path)
td.head()

,TD
AAPL,682.88
ADBE,424.50
ASML,544.30
PAYC,377.00
SHOP,306.36


In [39]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,686,172,8748,0,683
ABMD,541,1083,2165,0,0
ABNB,344,230,344,0,0
ADBE,0,0,0,0,424
ADYEY,5028,178,2855,114,0


In [40]:
out.to_csv(f'{DL}/clean.csv')

In [41]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_1,Value_2,Value_3
AAPL,858,2573,1715
ABMD,1083,812,0
ABNB,689,574,1148
ADBE,850,1275,0
ADYEY,665,1898,1444


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,858,2573,1715
ABMD,1083,812,0
ABNB,689,574,1148
ADBE,850,1275,0
ADYEY,665,1898,1444


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [15]:
!open $DL/mw.csv

In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*